In [29]:
import numpy as np
import tensorflow as tf
from tensorflow import keras #keras framework hai jo tensorflow k andar aata hai iska use kr k hum models define krte hai aur models ko save/ load krte hai
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers #isme neural network ki saari layers hoti hai 
import pandas as pd

In [30]:
df = pd.DataFrame({
    "soil_moisture": [0.10, 0.15, 0.20, 0.25, 0.40, 0.60, 0.35, 0.18,
                      0.45, 0.05, 0.80, 0.27, 0.55, 0.70, 0.12, 0.30],
    "temperature_c": [34, 30, 26, 22, 28, 30, 19, 22,
                      35, 24, 33, 33, 21, 25, 20, 29],
    "sunlight_hours": [9, 8, 7, 4, 8, 10, 3, 10,
                       12, 5, 9, 11, 2, 6, 1, 9],
    "needs_water": [1, 1, 1, 0, 0, 0, 0, 1,
                    0, 1, 0, 1, 0, 0, 1, 1]
})

In [31]:
df

,soil_moisture,temperature_c,sunlight_hours,needs_water
0,0.10,34,9,1
1,0.15,30,8,1
2,0.20,26,7,1
3,0.25,22,4,0
4,0.40,28,8,0
5,0.60,30,10,0
6,0.35,19,3,0
7,0.18,22,10,1
8,0.45,35,12,0
9,0.05,24,5,1


In [32]:
df.columns

Index(['soil_moisture', 'temperature_c', 'sunlight_hours', 'needs_water'], dtype='object')

In [33]:
X = df[['soil_moisture', 'temperature_c', 'sunlight_hours']]
y = df['needs_water']
# humne 2 dataframes/matrix bane, X me 3 cols aur Y me ek col save kiya

In [34]:
X_min = X.min()
X_max = X.max()
X_scaled = (X - X_min) / (X_max - X_min + 1e-8) #iss line me hum min-max scaling kr rhe hai(ya normalisation) iska motive hai humare dataset k saare features ko ek he range me le aana, jo usually 0 and 1 k beech hoti hai 
# why did we scale? Scaling karne se dono features 0 se 1 ki range mein aa jayenge, taaki model bina kisi bias ke dono ko barabar importance de sake aur training fast ho.
# Numerator (X - X_min): Ye step data ko "shift" karta hai taaki sabse chhota value 0 ban jaye.
# Denominator (X_max - X_min): Ye "range" (spread) hai. Isse divide karne se data "shrink" ho jata hai taaki sabse bada value 1 ban jaye.
# 1e-8 (Epsilon): Ye maths ka part nahi hai, ye coding safety hai. Agar kisi column mein saari values same huin (e.g., sab 10 hain), to X_max - X_min = 0 ho jayega. Maths mein 0 se divide karna error deta hai (Infinity). Isliye hum ek bohot chhota number (0.00000001) add kar dete hain taaki kabhi bhi denominator zero na ho.
# X.min(): Isne har column ka minimum nikal ke ek Series di.
# X - X_min: Pandas yahan Broadcasting use karta hai. Wo har column ke min value ko usi column ki har row se minus kar deta hai.

In [35]:
X_scaled

,soil_moisture,temperature_c,sunlight_hours
0,0.066667,0.9375,0.727273
1,0.133333,0.6875,0.636364
2,0.200000,0.4375,0.545455
3,0.266667,0.1875,0.272727
4,0.466667,0.5625,0.636364
5,0.733333,0.6875,0.818182
6,0.400000,0.0000,0.181818
7,0.173333,0.1875,0.818182
8,0.533333,1.0000,1.000000
9,0.000000,0.3125,0.363636


In [36]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.25, random_state=42, stratify=y
)
'''
-X_scaled: Aapke features (sawal) jo scale ho chuke hain.
-y: Aapke labels (jawab).
-test_size=0.25:
Iska matlab hai 25% data Test ke liye alag kar do aur bacha hua 75% Train ke liye use karo.
-random_state=42:
Jab computer random tukde karta hai, to wo har baar alag tukde kar sakta hai.
random_state=42 dene se hum computer ko "lock" kar dete hain ki "bhai, har baar same tareeke se hi random split karna".
Isse agar aap ya main ye code run karein, toh hum dono ka result exact same aayega. (Number 42 bas ek convention hai, aap 1, 100, kuch bhi likh sakte thay).
-stratify=y (Sabse Important):
Ye ensure karta hai ki aapke Train aur Test data mein Cases ka Ratio baraabar ho.
Example: Agar y (needs_water) mein:
60% plants ko paani chahiye (1).
40% plants ko paani nahi chahiye (0).
Agar hum stratify=y nahi likhte, to ho sakta hai ki "Test" data mein galti se sirf wo plants aa jayein jinhe paani chahiye. Model fail ho jayega.
stratify=y likhne se, computer ensure karega ki Test set mein bhi wahi 60:40 ratio bana rahe.
'''

'\n-X_scaled: Aapke features (sawal) jo scale ho chuke hain.\n-y: Aapke labels (jawab).\n-test_size=0.25:\nIska matlab hai 25% data Test ke liye alag kar do aur bacha hua 75% Train ke liye use karo.\n-random_state=42:\nJab computer random tukde karta hai, to wo har baar alag tukde kar sakta hai.\nrandom_state=42 dene se hum computer ko "lock" kar dete hain ki "bhai, har baar same tareeke se hi random split karna".\nIsse agar aap ya main ye code run karein, toh hum dono ka result exact same aayega. (Number 42 bas ek convention hai, aap 1, 100, kuch bhi likh sakte thay).\n-stratify=y (Sabse Important):\nYe ensure karta hai ki aapke Train aur Test data mein Cases ka Ratio baraabar ho.\nExample: Agar y (needs_water) mein:\n60% plants ko paani chahiye (1).\n40% plants ko paani nahi chahiye (0).\nAgar hum stratify=y nahi likhte, to ho sakta hai ki "Test" data mein galti se sirf wo plants aa jayein jinhe paani chahiye. Model fail ho jayega.\nstratify=y likhne se, computer ensure karega ki Tes

In [37]:
model = keras.Sequential([
    layers.Input(shape= (X_train.shape[1],)),
    layers.Dense(8,activation= 'relu'),
    layers.Dense(1,activation = 'sigmoid')
])
'''
1. model = keras.Sequential([...])
Sequential ka matlab hai "ek ke baad ek".
Ye ek aisa dibba (container) hai jisme aap layers ko line se jodte ho. Data pehli layer mein ghusega, process hoga, fir agli layer mein jayega, bilkul seedha raasta.

2. layers.Input(shape=(X_train.shape[1]))
Darwaza (Entry Gate): Ye network ka entry point hai.
X_train.shape[1] ka matlab hai humare features ki ginti (Columns).
Agar aapke paas 3 columns thay (Temperature, Moisture, Sunlight), toh yahan shape (3) hogi.
Ye layer model ko batati hai ki "Bhai, tayyar rehna, 3 values ka ek packet aane wala hai".

3. layers.Dense(8, activation='relu')
Hidden Layer: Ye network ka dimaag hai jahan wo seekhta hai.
Dense: Iska matlab "Fully Connected". Is layer ka har neuron pichli layer ke har neuron se juda hua hai.
8: Is layer mein 8 Neurons (chhote dimaag) hain. Humne 8 choose kiye taaki model thoda complex patterns seekh sake. (Ye number hum badal sakte hain experiment karke).
activation='relu': Ye ek switch ki tarah hai.
ReLU (Rectified Linear Unit): Ye kehta hai: "Agar value positive hai to aage jaane do, agar negative hai to zero kar do (band kar do)."
Ye model ko "Linear" (seedhi line) se hata kar complex cheezein seekhne mein madad karta hai.

4. layers.Dense(1, activation='sigmoid')
Output Layer: Ye aakhri layer hai jo faisla sunayegi.
1: Yahan sirf 1 Neuron hai. Kyun? Kyunki humein ek hi jawab chahiye: "Paani chahiye (1) ya nahi (0)".
activation='sigmoid':
Sigmoid function kisi bhi number ko 0 aur 1 ke beech mein changing probability mein badal deta hai.
Jaise: Output 0.8 aaya, toh iska matlab "80% chance hai ki paani chahiye".
Binary Classification (Yes/No problems) ke liye output layer mein hamesha Sigmoid use hota hai. 
'''

'\n1. model = keras.Sequential([...])\nSequential ka matlab hai "ek ke baad ek".\nYe ek aisa dibba (container) hai jisme aap layers ko line se jodte ho. Data pehli layer mein ghusega, process hoga, fir agli layer mein jayega, bilkul seedha raasta.\n\n2. layers.Input(shape=(X_train.shape[1]))\nDarwaza (Entry Gate): Ye network ka entry point hai.\nX_train.shape[1] ka matlab hai humare features ki ginti (Columns).\nAgar aapke paas 3 columns thay (Temperature, Moisture, Sunlight), toh yahan shape (3) hogi.\nYe layer model ko batati hai ki "Bhai, tayyar rehna, 3 values ka ek packet aane wala hai".\n\n3. layers.Dense(8, activation=\'relu\')\nHidden Layer: Ye network ka dimaag hai jahan wo seekhta hai.\nDense: Iska matlab "Fully Connected". Is layer ka har neuron pichli layer ke har neuron se juda hua hai.\n8: Is layer mein 8 Neurons (chhote dimaag) hain. Humne 8 choose kiye taaki model thoda complex patterns seekh sake. (Ye number hum badal sakte hain experiment karke).\nactivation=\'relu\':

In [ ]:
model.compile(optimizer='sgd',loss = 'binary_crossentropy',metrics= ['accuracy'])
# model.compile woh step hai jahan aap apne model ko training ke liye configure karte hain. Is step mein aap batate hain ki model seekhega kaise aur apni galtiyon ko sudharega kaise.
'''
1. optimizer='opt' (The Teacher)
Matlab: Optimizer woh algorithm hai jo model ke "weights" (dimaag ke neurons ke connections) ko update karta hai taaki error kam ho.
2. loss='binary_crossentropy' (The Error Calculator)
Matlab: Loss function model ko batata hai ki usne kitni buri prediction ki. Jitna zyada Loss, utna bura model. Model ka maqsad is "Loss" ko zero ke kareeb laana hai.
Choice:
'binary_crossentropy': Ye tab use hota hai jab aap Yes/No (0 ya 1) wala classification kar rahe hain (Binary Classification).
Agar multiple categories hoti (Cat, Dog, Mouse), toh hum 'categorical_crossentropy' use karte
3. metrics=['accuracy'] (The Report Card)
Matlab: Loss function machine ke samajhne ke liye hota hai (maths), par humans ko samajhne ke liye humein kuch aasaan chahiye, jaise "Accuracy".
accuracy batata hai: "100 mein se kitne sawal sahi kiye?" (e.g., 90% accuracy).
Syntax:
Notice karein ki ye ek List [...] ke andar hai ['accuracy'].
Aap yahan ek se zyada metrics bhi de sakte hain, jaise ['accuracy', 'precision', 'recall'].
'''

In [ ]:
# Yahan aap 3 alag-alag tarah ki Gradient Descent (learning techniques) use kar rahe hain, jinka main farq Batch Size (ek baar mein kitna data padhna hai) par depend karta hai.

history_full = model.fit(X_train, y_train, epochs=100, batch_size=len(X_train), verbose=1)

history = model.fit(
    X_train,y_train,
    validation_data = (X_test.values,y_test.values),
    epochs = 100,batch_size = 4 , verbose = 1
)

history_minibatch = model.fit(X_train, y_train, epochs=100, batch_size=100, verbose=1)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 596ms/step - accuracy: 0.4167 - loss: 0.7081
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.4167 - loss: 0.7081
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.4167 - loss: 0.7080
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.4167 - loss: 0.7080
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.4167 - loss: 0.7079
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.4167 - loss: 0.7079
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.4167 - loss: 0.7078
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.4167 - loss: 0.7078
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.4167 - loss: 0.7077
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.4167 - loss: 0.7077
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.4167 - loss: 0.7077
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.4167 - l

In [40]:
from tensorflow.keras import optimizers

In [41]:
opt = optimizers.SGD(learning_rate = 0.01, momentum = 0.9)